In [222]:
# while len(more) > 0:

#     # decide how many people the current case infects and add them to more
#     for i in range(select(spreadProbs)):
#         more.append(case)

# #     # decide if asymptomatic or not
# #     if decide(40) == 0:

#     # add current case to total # of cases
#     # (only sympotmatic cases adjust threshold)
#     sympCount += 1

#     # if case count is at or over threshold number
#     if sympCount > nthThresh(n, thresh):
#         # calculate new threshold
#         thresh = nthThresh(n, thresh)
#         # adjust spreadProbs weights and calculate r0 for corresponding threshold
#         thisThresh()
#         # add one to threshold n value
#         n += 1

#     # select which demographic the current case belongs to
#     demo = select(infectProbs)
#     weathering['demo'].append(demo)

#     # select which note
#     weathering['note'].append(midi(0))

#     # select incubation period (delay time)
#     delay = incubate(n)

#     # calculate time from beginning that symptoms occur for current case
#     if sympCount == 1:
#         time = delay
#     else:
#         time = delay + weathering['time'][more[0]]
#     weathering['time'].append(time)

#     # decide/record if case results in death or not
#     death = decide(deathProbs[demo])
#     weathering['death'].append(death)
    
#     weathering['n'].append(n)

#     case += 1

#     # remove current case from how many more cases left
#     more.pop(0)


In [223]:
# import dependencies
# import pandas as pd
# import numpy as np
import random
import json


In [224]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# display dictionary
stateData


{'tennessee': {'white': {'population': 5200531,
   'inverted_ccvi': 45.91,
   'chance_of_infection': 15.44,
   'chance_of_death': 1.47,
   'generated_cases': 0,
   'generated_deaths': 0},
  'black': {'population': 1120221,
   'inverted_ccvi': 28.16,
   'chance_of_infection': 15.65,
   'chance_of_death': 1.68,
   'generated_cases': 0,
   'generated_deaths': 0},
  'native': {'population': 18151,
   'inverted_ccvi': 40.66,
   'chance_of_infection': 9.31,
   'chance_of_death': 1.12,
   'generated_cases': 0,
   'generated_deaths': 0},
  'asian': {'population': 117348,
   'inverted_ccvi': 49.7,
   'chance_of_infection': 9.41,
   'chance_of_death': 0.79,
   'generated_cases': 0,
   'generated_deaths': 0},
  'pacific': {'population': 3735,
   'inverted_ccvi': 37.37,
   'chance_of_infection': 24.52,
   'chance_of_death': 1.03,
   'generated_cases': 0,
   'generated_deaths': 0},
  'hispanic': {'population': 363753,
   'inverted_ccvi': 32.45,
   'chance_of_infection': 25.67,
   'chance_of_death':

In [225]:
# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probability that an infection will result in death for each demographic
deathProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])


In [226]:
# function to determine how many cases correspond to a particular threshold
def nthThresh(n,t):
    return 100 * n + t

# function to adjust spreadProbs weights and calculate r0 for corresponding threshold
def thisThresh():
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    r0 = (spreadProbs[1] + 2*spreadProbs[2] + 3*spreadProbs[3]) / sum(spreadProbs)

# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        sub = 3
        return random.randint(0, sub*40)*2000/sub
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([6,8,5])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([6,8,5,7])
        return random.randint(0, sub*40)*2000/sub

# function to select a value based on a probability distribution array
def select(probsArray):
    return random.choices(list(range(len(probsArray))),weights=probsArray)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

# function to generate midi notes
def midi(t):
    note = random.choice(midiNotes)
    return note + t

In [227]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
times = [0]
# which case
case = 0
# array of midi notes
midiNotes = [48, 51, 55, 58, 62, 63, 65, 67, 68, 68, 74, 77, 78, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': [],
    'n': [],
}


In [228]:
while len(times) > 0:
    
    times = sorted(times)
    
    # add current case to total # of cases
        # (only sympotmatic cases adjust threshold)
    sympCount += 1
    
    # if case count is at or over threshold number
    if sympCount > nthThresh(n, thresh):
        # calculate new threshold
        thresh = nthThresh(n, thresh)
        # adjust spreadProbs weights and calculate r0 for corresponding threshold
        thisThresh()
        # add one to threshold n value
        n += 1

    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)): 
        
        
#         # decide if asymptomatic or not
#         if decide(40) == 0:

        # select which demographic the current case belongs to
        demo = select(infectProbs)
        weathering['demo'].append(demo)

        # select which note
        weathering['note'].append(midi(0))

        # select incubation period (delay time)
        delay = incubate(n)
        
        time = delay + times[0]
        
        weathering['time'].append(time)
        
        times.append(time)

        # decide/record if case results in death or not
        death = decide(deathProbs[demo])
        weathering['death'].append(death)

        weathering['n'].append(n)

    # remove current case from timestaps of remaining cases
    times.pop(0)


In [229]:
result = sorted(list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death'], weathering['n'])))

len(result)


11532

In [230]:
for i, r in enumerate(result):
    print(f'{i}: {r[4]}')



0: 1
1: 1
2: 1
3: 1
4: 1
5: 1
6: 1
7: 1
8: 1
9: 1
10: 1
11: 1
12: 1
13: 1
14: 1
15: 1
16: 1
17: 1
18: 1
19: 1
20: 1
21: 1
22: 1
23: 1
24: 1
25: 1
26: 1
27: 1
28: 1
29: 1
30: 1
31: 1
32: 1
33: 1
34: 1
35: 1
36: 1
37: 1
38: 1
39: 1
40: 1
41: 1
42: 1
43: 1
44: 1
45: 1
46: 1
47: 1
48: 1
49: 1
50: 1
51: 1
52: 1
53: 1
54: 1
55: 1
56: 1
57: 1
58: 1
59: 1
60: 1
61: 1
62: 1
63: 1
64: 1
65: 1
66: 1
67: 1
68: 1
69: 1
70: 1
71: 1
72: 1
73: 1
74: 1
75: 1
76: 1
77: 1
78: 1
79: 1
80: 1
81: 1
82: 1
83: 1
84: 1
85: 1
86: 1
87: 1
88: 1
89: 1
90: 1
91: 1
92: 1
93: 1
94: 1
95: 1
96: 1
97: 1
98: 1
99: 1
100: 1
101: 1
102: 1
103: 1
104: 1
105: 1
106: 2
107: 2
108: 1
109: 1
110: 1
111: 1
112: 1
113: 1
114: 1
115: 1
116: 2
117: 1
118: 1
119: 1
120: 1
121: 2
122: 1
123: 1
124: 1
125: 1
126: 2
127: 1
128: 1
129: 1
130: 1
131: 2
132: 1
133: 1
134: 2
135: 1
136: 1
137: 1
138: 1
139: 1
140: 1
141: 1
142: 1
143: 2
144: 1
145: 1
146: 1
147: 2
148: 2
149: 1
150: 1
151: 2
152: 1
153: 1
154: 1
155: 1
156: 2
157: 1
158:

1894: 3
1895: 2
1896: 4
1897: 2
1898: 4
1899: 6
1900: 4
1901: 4
1902: 4
1903: 3
1904: 2
1905: 4
1906: 4
1907: 6
1908: 6
1909: 5
1910: 4
1911: 5
1912: 6
1913: 5
1914: 3
1915: 4
1916: 2
1917: 5
1918: 3
1919: 6
1920: 4
1921: 3
1922: 5
1923: 3
1924: 2
1925: 6
1926: 5
1927: 6
1928: 3
1929: 4
1930: 3
1931: 4
1932: 4
1933: 6
1934: 4
1935: 5
1936: 4
1937: 3
1938: 6
1939: 4
1940: 5
1941: 3
1942: 4
1943: 2
1944: 5
1945: 5
1946: 4
1947: 4
1948: 5
1949: 6
1950: 4
1951: 4
1952: 4
1953: 5
1954: 4
1955: 5
1956: 2
1957: 6
1958: 5
1959: 5
1960: 4
1961: 6
1962: 3
1963: 3
1964: 5
1965: 4
1966: 5
1967: 4
1968: 5
1969: 5
1970: 4
1971: 3
1972: 2
1973: 5
1974: 2
1975: 3
1976: 2
1977: 2
1978: 5
1979: 6
1980: 6
1981: 6
1982: 3
1983: 6
1984: 6
1985: 4
1986: 4
1987: 3
1988: 4
1989: 2
1990: 4
1991: 2
1992: 3
1993: 4
1994: 3
1995: 3
1996: 4
1997: 5
1998: 6
1999: 3
2000: 5
2001: 6
2002: 4
2003: 2
2004: 2
2005: 3
2006: 2
2007: 5
2008: 4
2009: 4
2010: 5
2011: 5
2012: 6
2013: 2
2014: 3
2015: 2
2016: 2
2017: 5
2018: 2


3394: 5
3395: 6
3396: 6
3397: 3
3398: 4
3399: 6
3400: 7
3401: 6
3402: 5
3403: 7
3404: 8
3405: 4
3406: 8
3407: 5
3408: 7
3409: 5
3410: 3
3411: 3
3412: 8
3413: 7
3414: 5
3415: 7
3416: 7
3417: 5
3418: 6
3419: 5
3420: 5
3421: 7
3422: 7
3423: 5
3424: 7
3425: 6
3426: 7
3427: 6
3428: 7
3429: 6
3430: 6
3431: 3
3432: 5
3433: 4
3434: 6
3435: 8
3436: 8
3437: 5
3438: 8
3439: 4
3440: 7
3441: 8
3442: 6
3443: 5
3444: 8
3445: 5
3446: 3
3447: 6
3448: 6
3449: 8
3450: 7
3451: 6
3452: 6
3453: 6
3454: 7
3455: 4
3456: 6
3457: 6
3458: 3
3459: 5
3460: 3
3461: 6
3462: 6
3463: 6
3464: 6
3465: 5
3466: 7
3467: 7
3468: 5
3469: 7
3470: 4
3471: 8
3472: 7
3473: 7
3474: 7
3475: 7
3476: 6
3477: 6
3478: 8
3479: 8
3480: 4
3481: 3
3482: 5
3483: 5
3484: 3
3485: 4
3486: 4
3487: 5
3488: 5
3489: 3
3490: 8
3491: 8
3492: 6
3493: 6
3494: 3
3495: 4
3496: 5
3497: 3
3498: 4
3499: 4
3500: 7
3501: 7
3502: 5
3503: 6
3504: 8
3505: 4
3506: 6
3507: 3
3508: 3
3509: 4
3510: 3
3511: 7
3512: 4
3513: 6
3514: 5
3515: 6
3516: 5
3517: 6
3518: 8


5177: 4
5178: 6
5179: 5
5180: 6
5181: 7
5182: 10
5183: 4
5184: 6
5185: 10
5186: 8
5187: 5
5188: 6
5189: 6
5190: 8
5191: 10
5192: 5
5193: 6
5194: 5
5195: 6
5196: 8
5197: 9
5198: 8
5199: 6
5200: 5
5201: 6
5202: 10
5203: 5
5204: 5
5205: 10
5206: 5
5207: 7
5208: 8
5209: 6
5210: 9
5211: 5
5212: 9
5213: 4
5214: 8
5215: 10
5216: 7
5217: 4
5218: 10
5219: 8
5220: 10
5221: 7
5222: 9
5223: 6
5224: 10
5225: 6
5226: 7
5227: 10
5228: 6
5229: 8
5230: 9
5231: 4
5232: 7
5233: 7
5234: 8
5235: 4
5236: 4
5237: 6
5238: 6
5239: 4
5240: 8
5241: 5
5242: 5
5243: 4
5244: 7
5245: 5
5246: 8
5247: 9
5248: 10
5249: 8
5250: 10
5251: 10
5252: 7
5253: 8
5254: 8
5255: 5
5256: 5
5257: 9
5258: 10
5259: 8
5260: 6
5261: 6
5262: 8
5263: 9
5264: 4
5265: 8
5266: 5
5267: 10
5268: 6
5269: 7
5270: 5
5271: 8
5272: 8
5273: 6
5274: 4
5275: 8
5276: 6
5277: 10
5278: 6
5279: 6
5280: 6
5281: 9
5282: 10
5283: 5
5284: 7
5285: 10
5286: 6
5287: 5
5288: 8
5289: 4
5290: 4
5291: 9
5292: 9
5293: 7
5294: 7
5295: 8
5296: 4
5297: 6
5298: 9
5299: 

6893: 11
6894: 9
6895: 6
6896: 7
6897: 6
6898: 7
6899: 6
6900: 6
6901: 5
6902: 9
6903: 9
6904: 8
6905: 7
6906: 8
6907: 10
6908: 6
6909: 9
6910: 11
6911: 10
6912: 11
6913: 7
6914: 11
6915: 8
6916: 6
6917: 7
6918: 7
6919: 9
6920: 9
6921: 7
6922: 11
6923: 7
6924: 7
6925: 9
6926: 11
6927: 9
6928: 6
6929: 10
6930: 8
6931: 8
6932: 10
6933: 8
6934: 8
6935: 5
6936: 10
6937: 11
6938: 10
6939: 9
6940: 5
6941: 11
6942: 9
6943: 5
6944: 7
6945: 9
6946: 8
6947: 6
6948: 6
6949: 8
6950: 7
6951: 8
6952: 8
6953: 6
6954: 10
6955: 8
6956: 8
6957: 7
6958: 8
6959: 11
6960: 7
6961: 9
6962: 8
6963: 6
6964: 6
6965: 5
6966: 8
6967: 7
6968: 8
6969: 9
6970: 5
6971: 11
6972: 9
6973: 10
6974: 6
6975: 10
6976: 10
6977: 6
6978: 10
6979: 10
6980: 10
6981: 10
6982: 8
6983: 7
6984: 7
6985: 12
6986: 6
6987: 9
6988: 8
6989: 8
6990: 5
6991: 10
6992: 8
6993: 6
6994: 11
6995: 8
6996: 10
6997: 11
6998: 8
6999: 10
7000: 8
7001: 5
7002: 5
7003: 11
7004: 6
7005: 10
7006: 9
7007: 7
7008: 6
7009: 12
7010: 8
7011: 6
7012: 10
7013: 

8892: 9
8893: 13
8894: 7
8895: 11
8896: 10
8897: 8
8898: 11
8899: 12
8900: 8
8901: 9
8902: 13
8903: 8
8904: 10
8905: 10
8906: 8
8907: 8
8908: 9
8909: 9
8910: 7
8911: 10
8912: 8
8913: 8
8914: 12
8915: 10
8916: 10
8917: 10
8918: 7
8919: 7
8920: 7
8921: 8
8922: 7
8923: 11
8924: 7
8925: 9
8926: 9
8927: 7
8928: 10
8929: 9
8930: 10
8931: 8
8932: 11
8933: 12
8934: 8
8935: 11
8936: 9
8937: 7
8938: 7
8939: 11
8940: 10
8941: 9
8942: 9
8943: 12
8944: 10
8945: 11
8946: 8
8947: 11
8948: 9
8949: 8
8950: 7
8951: 11
8952: 11
8953: 8
8954: 10
8955: 8
8956: 10
8957: 7
8958: 12
8959: 12
8960: 11
8961: 9
8962: 9
8963: 7
8964: 7
8965: 8
8966: 10
8967: 8
8968: 10
8969: 8
8970: 8
8971: 8
8972: 8
8973: 7
8974: 11
8975: 11
8976: 11
8977: 8
8978: 13
8979: 9
8980: 13
8981: 13
8982: 8
8983: 8
8984: 12
8985: 10
8986: 8
8987: 8
8988: 7
8989: 10
8990: 11
8991: 12
8992: 12
8993: 7
8994: 11
8995: 10
8996: 7
8997: 8
8998: 8
8999: 11
9000: 10
9001: 8
9002: 10
9003: 8
9004: 13
9005: 9
9006: 10
9007: 8
9008: 9
9009: 7
901

10892: 12
10893: 11
10894: 10
10895: 13
10896: 11
10897: 10
10898: 12
10899: 10
10900: 12
10901: 13
10902: 12
10903: 11
10904: 11
10905: 12
10906: 13
10907: 13
10908: 11
10909: 13
10910: 11
10911: 11
10912: 10
10913: 11
10914: 12
10915: 10
10916: 12
10917: 10
10918: 12
10919: 12
10920: 14
10921: 11
10922: 14
10923: 12
10924: 11
10925: 11
10926: 11
10927: 12
10928: 13
10929: 11
10930: 12
10931: 10
10932: 10
10933: 12
10934: 11
10935: 10
10936: 10
10937: 10
10938: 14
10939: 10
10940: 12
10941: 12
10942: 10
10943: 13
10944: 12
10945: 13
10946: 10
10947: 11
10948: 12
10949: 10
10950: 10
10951: 10
10952: 11
10953: 10
10954: 13
10955: 12
10956: 12
10957: 12
10958: 13
10959: 13
10960: 10
10961: 10
10962: 12
10963: 13
10964: 10
10965: 12
10966: 11
10967: 11
10968: 11
10969: 11
10970: 13
10971: 10
10972: 13
10973: 11
10974: 10
10975: 11
10976: 13
10977: 11
10978: 13
10979: 12
10980: 11
10981: 10
10982: 12
10983: 11
10984: 13
10985: 11
10986: 13
10987: 11
10988: 10
10989: 11
10990: 13
10991: 11


In [231]:
timing = {}
ms = 0
infections = -1

for time in result:
    

    if time[0] != ms:
        infections += 1
        timing[infections] = {'demoNotes': [time[1], time[2], time[3]]}
        timing[infections]['delay'] = time[0] - ms
        ms = time[0]

    else:
        for i in range(1,4):
            timing[infections]['demoNotes'].append(time[i])

            
            
            
#     if time[0] != ms:
#         timing[time[0]] = {'demoNotes': [time[1], time[2], time[3]]}
#         timing[time[0]]['delay'] = time[0] - ms
#         ms = time[0]

#     else:
#         for i in range(1,4):
#             timing[time[0]]['demoNotes'].append(time[i])


In [232]:
timing



{0: {'demoNotes': [0, 74, 0], 'delay': 18666.666666666668},
 1: {'demoNotes': [5, 65, 0], 'delay': 20000.000000000004},
 2: {'demoNotes': [0, 74, 0], 'delay': 1333.3333333333358},
 3: {'demoNotes': [1, 77, 0], 'delay': 2000.0},
 4: {'demoNotes': [5, 58, 0], 'delay': 9333.333333333336},
 5: {'demoNotes': [5, 48, 0], 'delay': 7333.3333333333285},
 6: {'demoNotes': [2, 51, 0], 'delay': 1333.3333333333285},
 7: {'demoNotes': [3, 78, 0], 'delay': 666.6666666666715},
 8: {'demoNotes': [1, 68, 0], 'delay': 666.6666666666715},
 9: {'demoNotes': [5, 55, 0], 'delay': 2000.0},
 10: {'demoNotes': [1, 63, 0], 'delay': 1999.9999999999927},
 11: {'demoNotes': [1, 51, 0], 'delay': 1999.9999999999927},
 12: {'demoNotes': [5, 58, 0], 'delay': 3333.333333333343},
 13: {'demoNotes': [5, 63, 0, 5, 67, 0], 'delay': 666.6666666666715},
 14: {'demoNotes': [5, 77, 0], 'delay': 1333.3333333333285},
 15: {'demoNotes': [0, 77, 0], 'delay': 666.6666666666715},
 16: {'demoNotes': [0, 58, 0], 'delay': 1333.333333333